In [1]:
import pandas as pd
import nltk
import spacy
import string
import unicodedata
from collections import Counter
from itertools import chain

In [2]:
nltk.download('stopwords')
nltk.download('rslp')
!pip install spacy
!python -m spacy download pt_core_news_sm

[nltk_data] Downloading package stopwords to /home/pita/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /home/pita/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')


# Carregar dados

In [3]:
# Carregar dados

proposicoes = pd.read_csv("../dados/proposicoes.csv")

In [4]:
proposicoes

,texto,siglaTipo
0,"Altera a Lei nº 12.715, de 17 de setembro de 2...",MPV
1,Eleva para 15% a participação dos Municípios n...,PEC
2,"Altera a redação dos incisos II, III e VIII do...",PEC
3,Dispõe sobre a regulamentação da profissão de ...,PLS
4,"Altera as Leis nºs 11.482, de 31 de maio de 20...",PLS
5,"Altera a Medida Provisória nº 2.158-35, de 24 ...",PLS
6,Acrescenta os § 3º e 4º ao art. 22 da Lei nº 9...,PLS
7,"Altera a Lei n° 9.394, de 20 de dezembro de 19...",PLS
8,"Altera a Lei nº 8.742, de 07 de dezembro de 19...",PLS
9,Modifica o § 3º do art. 71 da Consolidação das...,PLS


# Transformar para minúsculas

In [5]:
# Transformar para minusculas
exemplo_texto = "Meu nome é Marcelo Pita"
exemplo_texto.lower()

'meu nome é marcelo pita'

In [6]:
# Transformar coluna texto para minusculas
proposicoes['texto'] = proposicoes['texto'].map(lambda x: x.lower())
proposicoes

,texto,siglaTipo
0,"altera a lei nº 12.715, de 17 de setembro de 2...",MPV
1,eleva para 15% a participação dos municípios n...,PEC
2,"altera a redação dos incisos ii, iii e viii do...",PEC
3,dispõe sobre a regulamentação da profissão de ...,PLS
4,"altera as leis nºs 11.482, de 31 de maio de 20...",PLS
5,"altera a medida provisória nº 2.158-35, de 24 ...",PLS
6,acrescenta os § 3º e 4º ao art. 22 da lei nº 9...,PLS
7,"altera a lei n° 9.394, de 20 de dezembro de 19...",PLS
8,"altera a lei nº 8.742, de 07 de dezembro de 19...",PLS
9,modifica o § 3º do art. 71 da consolidação das...,PLS


# Remoção de palavras por tamanho

In [7]:
# Remover palavras com menos de 3 e mais de 25 caracteres
exemplo_texto = "este é um exemplo de texto com sahhsdhidisdiasdjisajdijdisajd palavras pequenas e grandes"
palavras_adequadas = []
for p in exemplo_texto.split():
    tamanho_p = len(p)
    if tamanho_p >= 3 and tamanho_p <= 25:
        palavras_adequadas.append(p)
exemplo_texto_ok = " ".join(palavras_adequadas)
exemplo_texto_ok

'este exemplo texto com palavras pequenas grandes'

In [8]:
proposicoes_tamanho_ok = []

for index, row in proposicoes.iterrows():
    texto = row[0]
    palavras_adequadas = []
    for p in texto.split():
        tamanho_p = len(p)
        if tamanho_p >= 3 and tamanho_p <= 25:
            palavras_adequadas.append(p)
    proposicoes_tamanho_ok.append(" ".join(palavras_adequadas))

proposicoes['texto'] = proposicoes_tamanho_ok
proposicoes

,texto,siglaTipo
0,"altera lei 12.715, setembro 2012, que institui...",MPV
1,eleva para 15% participação dos municípios par...,PEC
2,"altera redação dos incisos ii, iii viii art. 1...",PEC
3,dispõe sobre regulamentação profissão arqueólo...,PLS
4,"altera leis nºs 11.482, maio 2007, 7.713, deze...",PLS
5,"altera medida provisória 2.158-35, agosto 2001...",PLS
6,"acrescenta art. lei 9.636, maio 1998, para que...",PLS
7,"altera lei 9.394, dezembro 1996, que estabelec...",PLS
8,"altera lei 8.742, dezembro 1993, lei orgânica ...",PLS
9,modifica art. consolidação das leis trabalho c...,PLS


# Remoção de stopwords

In [9]:
# Carregar stopwords
stopwords = nltk.corpus.stopwords.words('portuguese')
stopwords

['de',
 'a',
 'o',
 'que',
 'e',
 'é',
 'do',
 'da',
 'em',
 'um',
 'para',
 'com',
 'não',
 'uma',
 'os',
 'no',
 'se',
 'na',
 'por',
 'mais',
 'as',
 'dos',
 'como',
 'mas',
 'ao',
 'ele',
 'das',
 'à',
 'seu',
 'sua',
 'ou',
 'quando',
 'muito',
 'nos',
 'já',
 'eu',
 'também',
 'só',
 'pelo',
 'pela',
 'até',
 'isso',
 'ela',
 'entre',
 'depois',
 'sem',
 'mesmo',
 'aos',
 'seus',
 'quem',
 'nas',
 'me',
 'esse',
 'eles',
 'você',
 'essa',
 'num',
 'nem',
 'suas',
 'meu',
 'às',
 'minha',
 'numa',
 'pelos',
 'elas',
 'qual',
 'nós',
 'lhe',
 'deles',
 'essas',
 'esses',
 'pelas',
 'este',
 'dele',
 'tu',
 'te',
 'vocês',
 'vos',
 'lhes',
 'meus',
 'minhas',
 'teu',
 'tua',
 'teus',
 'tuas',
 'nosso',
 'nossa',
 'nossos',
 'nossas',
 'dela',
 'delas',
 'esta',
 'estes',
 'estas',
 'aquele',
 'aquela',
 'aqueles',
 'aquelas',
 'isto',
 'aquilo',
 'estou',
 'está',
 'estamos',
 'estão',
 'estive',
 'esteve',
 'estivemos',
 'estiveram',
 'estava',
 'estávamos',
 'estavam',
 'estivera'

In [10]:
# Exemplo
exemplo_texto = "este é um texto com stopwords"
palavras_adequadas = [p for p in exemplo_texto.split() if p not in stopwords]
exemplo_texto_ok = " ".join(palavras_adequadas)
exemplo_texto_ok

'texto stopwords'

In [11]:
proposicoes_stopwords_ok = []

for index, row in proposicoes.iterrows():
    texto = row[0]
    palavras_adequadas = [p for p in texto.split() if p not in stopwords]
    proposicoes_stopwords_ok.append(" ".join(palavras_adequadas))

proposicoes['texto'] = proposicoes_stopwords_ok
proposicoes

,texto,siglaTipo
0,"altera lei 12.715, setembro 2012, institui pro...",MPV
1,eleva 15% participação municípios partilha arr...,PEC
2,"altera redação incisos ii, iii viii art. 142, ...",PEC
3,dispõe sobre regulamentação profissão arqueólo...,PLS
4,"altera leis nºs 11.482, maio 2007, 7.713, deze...",PLS
5,"altera medida provisória 2.158-35, agosto 2001...",PLS
6,"acrescenta art. lei 9.636, maio 1998, municípi...",PLS
7,"altera lei 9.394, dezembro 1996, estabelece di...",PLS
8,"altera lei 8.742, dezembro 1993, lei orgânica ...",PLS
9,modifica art. consolidação leis trabalho clt a...,PLS


# Remover pontuação

In [12]:
nlp = spacy.load("pt_core_news_sm")

In [13]:
pontuacao = string.punctuation.strip()
pontuacao_lista = []
for pont in pontuacao.strip():
    pontuacao_lista.append(pont)
pontuacao_lista.append('º')
pontuacao_lista.append('ª')
pontuacao_lista.append('...')
pontuacao_lista.append('“')
pontuacao_lista.append('”')
pontuacao_lista

['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~',
 'º',
 'ª',
 '...',
 '“',
 '”']

In [14]:
exemplo_texto = "ei! você aí... não acha isso um absurdo?!"
exemplo_texto_tokens = nlp(exemplo_texto)
exemplo_texto_tokens = [str(t) for t in exemplo_texto_tokens if str(t) not in pontuacao_lista]
exemplo_texto_ok = " ".join(exemplo_texto_tokens)
exemplo_texto_ok

'ei você aí não acha isso um absurdo'

In [15]:
proposicoes_ok = []
for index, row in proposicoes.iterrows():
    texto = row[0]
    texto_tokens = nlp(texto)
    texto_tokens = [str(t) for t in texto_tokens if str(t) not in pontuacao_lista]
    proposicoes_ok.append(" ".join(texto_tokens))
proposicoes['texto'] = proposicoes_ok
proposicoes

,texto,siglaTipo
0,altera lei 12.715 setembro 2012 institui progr...,MPV
1,eleva 15 participação municípios partilha arre...,PEC
2,altera redação incisos ii iii viii art 142 con...,PEC
3,dispõe sobre regulamentação profissão arqueólo...,PLS
4,altera leis nºs 11.482 maio 2007 7.713 dezembr...,PLS
5,altera medida provisória 2. 158-35 agosto 2001...,PLS
6,acrescenta art lei 9.636 maio 1998 municípios ...,PLS
7,altera lei 9.394 dezembro 1996 estabelece dire...,PLS
8,altera lei 8.742 dezembro 1993 lei orgânica as...,PLS
9,modifica art consolidação leis trabalho clt ap...,PLS


# Estemização

In [16]:
estemizador_port = nltk.stem.RSLPStemmer()

In [17]:
exemplo_texto = "água mole pedra dura tanto bate até que fura pedir"
palavras_estemizadas = []
for p in exemplo_texto.split():
    palavras_estemizadas.append(estemizador_port.stem(p))
palavras_estemizadas

['águ', 'mol', 'pedr', 'dur', 'tant', 'bat', 'até', 'que', 'fur', 'ped']

In [18]:
proposicoes_estemizadas = []
for index, row in proposicoes.iterrows():
    texto = row[0]
    texto_tokens_estemizados = [estemizador_port.stem(t) for t in texto.split()]
    proposicoes_estemizadas.append(" ".join(texto_tokens_estemizados))
proposicoes['texto_estemizado'] = proposicoes_estemizadas
proposicoes

,texto,siglaTipo,texto_estemizado
0,altera lei 12.715 setembro 2012 institui progr...,MPV,alt lei 12.715 setembr 2012 institu program in...
1,eleva 15 participação municípios partilha arre...,PEC,elev 15 particip municípi partilh arrecad cont...
2,altera redação incisos ii iii viii art 142 con...,PEC,alt red incis ii iii vii art 142 constitu fede...
3,dispõe sobre regulamentação profissão arqueólo...,PLS,dispõ sobr regulament profiss arqueólog outr p...
4,altera leis nºs 11.482 maio 2007 7.713 dezembr...,PLS,alt lei nº 11.482 mai 2007 7.713 dezembr 1988 ...
5,altera medida provisória 2. 158-35 agosto 2001...,PLS,alt med provisór 2. 158-35 agost 2001 confer e...
6,acrescenta art lei 9.636 maio 1998 municípios ...,PLS,acrescent art lei 9.636 mai 1998 municípi auto...
7,altera lei 9.394 dezembro 1996 estabelece dire...,PLS,alt lei 9.394 dezembr 1996 estabelec diretriz ...
8,altera lei 8.742 dezembro 1993 lei orgânica as...,PLS,alt lei 8.742 dezembr 1993 lei orgân assist so...
9,modifica art consolidação leis trabalho clt ap...,PLS,modif art consolid lei trabalh clt aprov decre...


# Lematização

In [19]:
exemplo_texto = "água mole pedra dura tanto bate até que fura"
lemas = []
exemplo_texto_tokens = nlp(exemplo_texto)
for t in exemplo_texto_tokens:
    lemas.append(t.lemma_)
lemas

['água', 'mole', 'pedrar', 'duro', 'tanto', 'bater', 'até', 'que', 'furar']

In [20]:
proposicoes_lemas = []
for index, row in proposicoes.iterrows():
    texto = row[0]
    texto_tokens = nlp(texto)
    lemas = []
    for t in texto_tokens:
        lemas.append(t.lemma_)
    proposicoes_lemas.append(" ".join(lemas))
proposicoes['texto_lemas'] = proposicoes_lemas
proposicoes

,texto,siglaTipo,texto_estemizado,texto_lemas
0,altera lei 12.715 setembro 2012 institui progr...,MPV,alt lei 12.715 setembr 2012 institu program in...,alterar lei 12.715 setembro 2012 instituir pro...
1,eleva 15 participação municípios partilha arre...,PEC,elev 15 particip municípi partilh arrecad cont...,elevar 15 participação município partilhar arr...
2,altera redação incisos ii iii viii art 142 con...,PEC,alt red incis ii iii vii art 142 constitu fede...,alterar redação inciso i iii viii art 142 cons...
3,dispõe sobre regulamentação profissão arqueólo...,PLS,dispõ sobr regulament profiss arqueólog outr p...,dispor sobrar regulamentação profissão arqueól...
4,altera leis nºs 11.482 maio 2007 7.713 dezembr...,PLS,alt lei nº 11.482 mai 2007 7.713 dezembr 1988 ...,alterar lei nºs 11.482 maio 2007 7.713 dezembr...
5,altera medida provisória 2. 158-35 agosto 2001...,PLS,alt med provisór 2. 158-35 agost 2001 confer e...,alterar medir provisório 2 . 158-35 agostar 20...
6,acrescenta art lei 9.636 maio 1998 municípios ...,PLS,acrescent art lei 9.636 mai 1998 municípi auto...,acrescentar art lei 9.636 maio 1998 município ...
7,altera lei 9.394 dezembro 1996 estabelece dire...,PLS,alt lei 9.394 dezembr 1996 estabelec diretriz ...,alterar lei 9.394 dezembro 1996 estabelecer di...
8,altera lei 8.742 dezembro 1993 lei orgânica as...,PLS,alt lei 8.742 dezembr 1993 lei orgân assist so...,alterar lei 8.742 dezembro 1993 lei orgânico a...
9,modifica art consolidação leis trabalho clt ap...,PLS,modif art consolid lei trabalh clt aprov decre...,modificar art consolidação lei trabalhar clt a...


# Remoção de acentuação

In [169]:
exemplo_texto = "água mole pedra dura tanto bate até que fura"
unicodedata.normalize('NFKD', exemplo_texto).encode('ASCII', 'ignore')

b'agua mole pedra dura tanto bate ate que fura'

In [ ]:
proposicoes['texto'] = proposicoes['texto'].map(lambda x: unicodedata.normalize('NFKD', x).encode('ASCII', 'ignore'))

In [173]:
proposicoes['texto_estemizado'] = proposicoes['texto_estemizado'].map(lambda x: unicodedata.normalize('NFKD', x).encode('ASCII', 'ignore'))

In [175]:
proposicoes['texto_lemas'] = proposicoes['texto_lemas'].map(lambda x: unicodedata.normalize('NFKD', x).encode('ASCII', 'ignore'))

In [207]:
proposicoes['texto'] = proposicoes['texto'].str.decode('utf-8')
proposicoes['texto_estemizado'] = proposicoes['texto_estemizado'].str.decode('utf-8')
proposicoes['texto_lemas'] = proposicoes['texto_lemas'].str.decode('utf-8')

In [208]:
proposicoes

,texto,siglaTipo,texto_estemizado,texto_lemas
0,altera lei 12.715 setembro 2012 institui progr...,MPV,alt lei 12.715 setembr 2012 institu program in...,alterar lei 12.715 setembro 2012 instituir pro...
1,eleva 15 participacao municipios partilha arre...,PEC,elev 15 particip municipi partilh arrecad cont...,elevar 15 participacao municipio partilhar arr...
2,altera redacao incisos ii iii viii art 142 con...,PEC,alt red incis ii iii vii art 142 constitu fede...,alterar redacao inciso i iii viii art 142 cons...
3,dispoe sobre regulamentacao profissao arqueolo...,PLS,dispo sobr regulament profiss arqueolog outr p...,dispor sobrar regulamentacao profissao arqueol...
4,altera leis nos 11.482 maio 2007 7.713 dezembr...,PLS,alt lei no 11.482 mai 2007 7.713 dezembr 1988 ...,alterar lei nos 11.482 maio 2007 7.713 dezembr...
5,altera medida provisoria 2. 158-35 agosto 2001...,PLS,alt med provisor 2. 158-35 agost 2001 confer e...,alterar medir provisorio 2 . 158-35 agostar 20...
6,acrescenta art lei 9.636 maio 1998 municipios ...,PLS,acrescent art lei 9.636 mai 1998 municipi auto...,acrescentar art lei 9.636 maio 1998 municipio ...
7,altera lei 9.394 dezembro 1996 estabelece dire...,PLS,alt lei 9.394 dezembr 1996 estabelec diretriz ...,alterar lei 9.394 dezembro 1996 estabelecer di...
8,altera lei 8.742 dezembro 1993 lei organica as...,PLS,alt lei 8.742 dezembr 1993 lei organ assist so...,alterar lei 8.742 dezembro 1993 lei organico a...
9,modifica art consolidacao leis trabalho clt ap...,PLS,modif art consolid lei trabalh clt aprov decre...,modificar art consolidacao lei trabalhar clt a...


# Remoção de palavras de baixa frequência

In [21]:
proposicoes

,texto,siglaTipo,texto_estemizado,texto_lemas
0,altera lei 12.715 setembro 2012 institui progr...,MPV,alt lei 12.715 setembr 2012 institu program in...,alterar lei 12.715 setembro 2012 instituir pro...
1,eleva 15 participação municípios partilha arre...,PEC,elev 15 particip municípi partilh arrecad cont...,elevar 15 participação município partilhar arr...
2,altera redação incisos ii iii viii art 142 con...,PEC,alt red incis ii iii vii art 142 constitu fede...,alterar redação inciso i iii viii art 142 cons...
3,dispõe sobre regulamentação profissão arqueólo...,PLS,dispõ sobr regulament profiss arqueólog outr p...,dispor sobrar regulamentação profissão arqueól...
4,altera leis nºs 11.482 maio 2007 7.713 dezembr...,PLS,alt lei nº 11.482 mai 2007 7.713 dezembr 1988 ...,alterar lei nºs 11.482 maio 2007 7.713 dezembr...
5,altera medida provisória 2. 158-35 agosto 2001...,PLS,alt med provisór 2. 158-35 agost 2001 confer e...,alterar medir provisório 2 . 158-35 agostar 20...
6,acrescenta art lei 9.636 maio 1998 municípios ...,PLS,acrescent art lei 9.636 mai 1998 municípi auto...,acrescentar art lei 9.636 maio 1998 município ...
7,altera lei 9.394 dezembro 1996 estabelece dire...,PLS,alt lei 9.394 dezembr 1996 estabelec diretriz ...,alterar lei 9.394 dezembro 1996 estabelecer di...
8,altera lei 8.742 dezembro 1993 lei orgânica as...,PLS,alt lei 8.742 dezembr 1993 lei orgân assist so...,alterar lei 8.742 dezembro 1993 lei orgânico a...
9,modifica art consolidação leis trabalho clt ap...,PLS,modif art consolid lei trabalh clt aprov decre...,modificar art consolidação lei trabalhar clt a...


In [22]:
frequencia_minima = 5

In [23]:
proposicoes_tokens = proposicoes['texto'].str.split().tolist()
c = Counter(chain.from_iterable(proposicoes_tokens))
proposicoes['texto'] = [' '.join([j for j in i if c[j] > frequencia_minima]) for i in proposicoes_tokens]

In [24]:
proposicoes_tokens = proposicoes['texto_estemizado'].str.split().tolist()
c = Counter(chain.from_iterable(proposicoes_tokens))
proposicoes['texto_estemizado'] = [' '.join([j for j in i if c[j] > frequencia_minima]) for i in proposicoes_tokens]

In [25]:
proposicoes_tokens = proposicoes['texto_lemas'].str.split().tolist()
c = Counter(chain.from_iterable(proposicoes_tokens))
proposicoes['texto_lemas'] = [' '.join([j for j in i if c[j] > frequencia_minima]) for i in proposicoes_tokens]

In [26]:
proposicoes

,texto,siglaTipo,texto_estemizado,texto_lemas
0,altera lei setembro 2012 institui programa inc...,MPV,alt lei setembr 2012 institu program incen ino...,alterar lei setembro 2012 instituir programar ...
1,participação municípios partilha arrecadação c...,PEC,elev particip municípi partilh arrecad contrib...,elevar participação município partilhar arreca...
2,altera redação incisos ii iii viii art 142 con...,PEC,alt red incis ii iii vii art 142 constitu fede...,alterar redação inciso i iii viii art 142 cons...
3,dispõe sobre regulamentação profissão outras p...,PLS,dispõ sobr regulament profiss outr provid,dispor sobrar regulamentação profissão outro p...
4,altera leis nºs 11.482 maio 2007 7.713 dezembr...,PLS,alt lei nº 11.482 mai 2007 7.713 dezembr 1988 ...,alterar lei nºs 11.482 maio 2007 7.713 dezembr...
5,altera medida provisória 2. agosto 2001 confer...,PLS,alt med provisór 2. agost 2001 confer export c...,alterar medir provisório 2 . agostar 2001 conf...
6,acrescenta art lei 9.636 maio 1998 municípios ...,PLS,acrescent art lei 9.636 mai 1998 municípi auto...,acrescentar art lei 9.636 maio 1998 município ...
7,altera lei 9.394 dezembro 1996 estabelece dire...,PLS,alt lei 9.394 dezembr 1996 estabelec diretriz ...,alterar lei 9.394 dezembro 1996 estabelecer di...
8,altera lei 8.742 dezembro 1993 lei orgânica as...,PLS,alt lei 8.742 dezembr 1993 lei orgân assist so...,alterar lei 8.742 dezembro 1993 lei orgânico a...
9,modifica art consolidação leis trabalho clt ap...,PLS,modif art consolid lei trabalh clt aprov decre...,modificar art consolidação lei trabalhar clt a...


In [29]:
proposicoes.filter(items=['texto']).to_csv("../dados/proposicoes_normalizadas.csv", sep=',',
                                           index=False, encoding='utf-8')